# Analysis

## Setup

Zuerst laden wir die benötigten Daten herunter und initialisieren die genutzten Python Objekte.

In [1]:
tables = [
    "careplans",
    "conditions",
    "observations",
    "patients",
]

files = [
    "data/others/",
    "data/asthma/",
    "data/gallstones/",
    "data/hypertension/",
]

In [2]:
!mkdir -p data/allergy

Syntaxfehler.


In [3]:
from urllib.request import urlopen
import os

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        print("File {} already exists, skipping download".format(filename))
    else:
        print("Downloading {}".format(filename))
        download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

for file in files:
    for table in tables:
        ensure_file_has_been_downloaded(file+table+".csv")

File data/others/careplans.csv already exists, skipping download
File data/others/conditions.csv already exists, skipping download
File data/others/observations.csv already exists, skipping download
File data/others/patients.csv already exists, skipping download
File data/asthma/careplans.csv already exists, skipping download
File data/asthma/conditions.csv already exists, skipping download
File data/asthma/observations.csv already exists, skipping download
File data/asthma/patients.csv already exists, skipping download
File data/gallstones/careplans.csv already exists, skipping download
File data/gallstones/conditions.csv already exists, skipping download
File data/gallstones/observations.csv already exists, skipping download
File data/gallstones/patients.csv already exists, skipping download
File data/hypertension/careplans.csv already exists, skipping download
File data/hypertension/conditions.csv already exists, skipping download
File data/hypertension/observations.csv already exis

In [4]:
from logger import Logger
from test_executer import TestExecutor

logger = Logger()
testExecutor = TestExecutor(logger)

## Mit Datenbank verbinden

In [5]:
import extract
import time

databaseFile = "data.sqlite"

logger.startTimeMeasurement('open-db', 'Connected to db and created tables')
connection = extract.connect_to_db(logger, databaseFile)  # create table patients, observations, conditions, careplans
logger.endTimeMeasurement('open-db')

In [6]:
def test_sqliteConnection(_logger):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablesInDb = list(map(lambda tableResult: tableResult[0], cursor.fetchall()))
    tablesInDb.sort()

    for table in tables:
        if not(table in tablesInDb):
            raise Exception('Table not found:', table)

testExecutor.execute('Test connection to database', test_sqliteConnection)

{"type": "info", "time": 1656406551774, "message": "✅ Test ran successfully: Test connection to database", "params": null}


## Daten in Datenbank laden

Lade der verwendete Daten in die Datenbank:

-   careplans
-   conditions
-   observations
-   patients

In [7]:
logger.startTimeMeasurement('load-data', 'Loading data into db')
for file in files:
    for table in tables:
        extract.insert_values_to_table(logger, connection.cursor(), table, "../"+ file + table + ".csv")  # TODO: insert ALL values in the right tables 
        connection.commit()
    logger.endTimeMeasurement('load-data')

{"type": "info", "time": 1656406551863, "message": "🏗 Extracting data from ../data/others/careplans.csv", "params": null}
{"type": "info", "time": 1656406551872, "message": "🏗 Extracting data from ../data/others/conditions.csv", "params": null}
{"type": "info", "time": 1656406551916, "message": "🏗 Extracting data from ../data/others/observations.csv", "params": null}
{"type": "info", "time": 1656406552608, "message": "🏗 Extracting data from ../data/others/patients.csv", "params": null}
{"type": "info", "time": 1656406552616, "message": "🏗 Extracting data from ../data/asthma/careplans.csv", "params": null}
{"type": "info", "time": 1656406552636, "message": "🏗 Extracting data from ../data/asthma/conditions.csv", "params": null}
{"type": "info", "time": 1656406552765, "message": "🏗 Extracting data from ../data/asthma/observations.csv", "params": null}
{"type": "info", "time": 1656406555241, "message": "🏗 Extracting data from ../data/asthma/patients.csv", "params": null}
{"type": "info", "

## Messung der Datenfehler

Für unsere Forschungsfrage sind nur alle Daten mit gemessenen BMI relevant. Wurde dieser nicht vermessen oder eingetragen können die Daten für die Forschungsfrage nicht verwendet werden und sind somit unbrauchbar.

In [8]:
all_patients_query = """
SELECT COUNT(id) FROM patients;"""
count_bmi_query = """
SELECT COUNT(distinct id) FROM patients JOIN observations on patients.id == observations.patient WHERE observations.Code = '59576-9'"""

count_all_bmi_query = f"""
SELECT COUNT(patient) FROM observations WHERE observations.Code = '59576-9'"""

patient_all_count = connection.execute(all_patients_query).fetchall()[0][0]
patient_bmi_count = connection.execute(count_bmi_query).fetchall()[0][0]
bmi_count = connection.execute(count_all_bmi_query).fetchall()[0][0]
ratio = round(patient_bmi_count/patient_all_count, 3) * 100

logger.log(f"Total num of patients {patient_all_count}.")
logger.log(f"Found {patient_bmi_count} patients ({ratio}%) with {bmi_count} BMI-values.")

{"type": "info", "time": 1656406561453, "message": "Total num of patients 1326.", "params": null}
{"type": "info", "time": 1656406561453, "message": "Found 450 patients (33.900000000000006%) with 3539 BMI-values.", "params": null}


In [9]:
import pandas as pd
cursor = connection.cursor()

In [11]:
# print tables of the database
cursor.execute('SELECT name FROM sqlite_master where type="table"') 
print(cursor.fetchall())

[('patients',), ('careplans',), ('conditions',), ('observations',)]


# Star Schema

#### Erstellen Dimensionstabellen

Tabelle patients_

In [12]:
# table patients_
cursor.execute('''DROP TABLE IF EXISTS patients_;''')
cursor.execute('''
        CREATE TABLE patients_ ( 
        ID STRING PRIMARY KEY UNIQUE,
        BIRTHDATE DATE,
        RACE STRING, 
        ETHNICITY STRING,
        GENDER STRING
        );''')
   
cursor.execute('INSERT INTO patients_ (ID, BIRTHDATE, RACE, ETHNICITY, GENDER) SELECT ID, BIRTHDATE, RACE, ETHNICITY, GENDER FROM PATIENTS;')

print(pd.read_sql_query("PRAGMA table_info(patients_)", connection))

   cid       name    type  notnull dflt_value  pk
0    0         ID  STRING        0       None   1
1    1  BIRTHDATE    DATE        0       None   0
2    2       RACE  STRING        0       None   0
3    3  ETHNICITY  STRING        0       None   0
4    4     GENDER  STRING        0       None   0


Tabelle observations_

In [13]:
# table observations_
cursor.execute('''DROP TABLE IF EXISTS observations_;''')
cursor.execute('''
        CREATE TABLE observations_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING,
        UNITS STRING, 
        TYPE STRING
        );''')

# create df observation
cursor.execute('''SELECT CODE, DESCRIPTION, UNITS, TYPE FROM OBSERVATIONS;''')
df_observations = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION', 'UNITS', 'TYPE'])

df_observations = df_observations.drop_duplicates(subset='CODE')

# transform dt in table
df_observations.to_sql('df_observations', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO observations_ (CODE, DESCRIPTION, UNITS, TYPE) SELECT CODE, DESCRIPTION, UNITS, TYPE FROM df_observations;''')

cursor.execute('''DROP TABLE IF EXISTS df_observations;''')
print(pd.read_sql_query("PRAGMA table_info(observations_)", connection))
#print(pd.read_sql_query("SELECT * FROM observations_", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0
2    2        UNITS  STRING        0       None   0
3    3         TYPE  STRING        0       None   0


Tabelle careplans_code

In [14]:
# table careplans_code
cursor.execute('''DROP TABLE IF EXISTS careplans_code;''')
cursor.execute('''
        CREATE TABLE careplans_code ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT CODE, DESCRIPTION FROM CAREPLANS;''')
df_careplans_code = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_careplans_code = df_careplans_code.drop_duplicates(subset='CODE')

# transform dt in table
df_careplans_code.to_sql('df_careplans_code', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO careplans_code (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_careplans_code;''')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_code;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_code)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_code", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0


Tabelle careplans_reasoncode

In [15]:
# table careplans_reasoncode
cursor.execute('''DROP TABLE IF EXISTS careplans_reasoncode;''')
cursor.execute('''
        CREATE TABLE careplans_reasoncode ( 
        REASONCODE STRING PRIMARY KEY UNIQUE,
        REASONDESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT REASONCODE, REASONDESCRIPTION FROM CAREPLANS;''')
df_careplans_reasoncode = pd.DataFrame(cursor.fetchall(), columns=['REASONCODE','REASONDESCRIPTION'])

df_careplans_reasoncode = df_careplans_reasoncode.drop_duplicates(subset='REASONCODE')

# transform dt in table
df_careplans_reasoncode.to_sql('df_careplans_reasoncode', connection, if_exists='replace', index=False)
   
cursor.execute('INSERT INTO careplans_reasoncode (REASONCODE, REASONDESCRIPTION) SELECT REASONCODE, REASONDESCRIPTION FROM df_careplans_reasoncode;')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_reasoncode;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_reasoncode)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_reasoncode", connection))

   cid               name    type  notnull dflt_value  pk
0    0         REASONCODE  STRING        0       None   1
1    1  REASONDESCRIPTION  STRING        0       None   0


Tabelle conditions_

In [16]:
# table conditions
cursor.execute('''DROP TABLE IF EXISTS conditions_;''')
cursor.execute('''
        CREATE TABLE conditions_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df conditions
cursor.execute('''SELECT CODE, DESCRIPTION FROM CONDITIONS;''')
df_conditions = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_conditions = df_conditions.drop_duplicates(subset='CODE')

# transform dt in table
df_conditions.to_sql('df_conditions', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO conditions_ (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_conditions;')

cursor.execute('''DROP TABLE IF EXISTS df_conditions;''')
print(pd.read_sql_query("PRAGMA table_info(conditions_)", connection))
#print(pd.read_sql_query("SELECT * FROM conditions_", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0


#### Erstelle Faktentabelle

In [17]:
cursor.execute('''DROP TABLE IF EXISTS fact_table;''')
cursor.execute('''
        CREATE TABLE fact_table ( 
        PATIENT_ID STRING,
        OBSERVATION_CODE STRING,
        VALUE STRING,
        DATE DATE,
        CONDITIONS_CODE STRING,
        ENDDATE DATE,
        CAREPLANS_CODE STRING,
        CAREPLANS_REASONCODE STRING,
        FOREIGN KEY (PATIENT_ID)
            REFERENCES patients_(ID),
        FOREIGN KEY (OBSERVATION_CODE)
            REFERENCES observations_(CODE),
        FOREIGN KEY (CONDITIONS_CODE)
            REFERENCES conditions_(CODE),
        FOREIGN KEY (CAREPLANS_CODE)
            REFERENCES careplans_code(CODE),
        FOREIGN KEY (CAREPLANS_REASONCODE)
            REFERENCES careplans_reasoncode(REASONCODE)
        );''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, OBSERVATION_CODE, VALUE, DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_REASONCODE, DATE, ENDDATE) 
                    SELECT PATIENT, REASONCODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CONDITIONS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CONDITIONS
                    ;''')

In [18]:
print(pd.read_sql_query("PRAGMA foreign_key_list(fact_table)", connection))

   id  seq                 table                  from          to  on_update  \
0   0    0  careplans_reasoncode  CAREPLANS_REASONCODE  REASONCODE  NO ACTION   
1   1    0        careplans_code        CAREPLANS_CODE        CODE  NO ACTION   
2   2    0           conditions_       CONDITIONS_CODE        CODE  NO ACTION   
3   3    0         observations_      OBSERVATION_CODE        CODE  NO ACTION   
4   4    0             patients_            PATIENT_ID          ID  NO ACTION   

   on_delete match  
0  NO ACTION  NONE  
1  NO ACTION  NONE  
2  NO ACTION  NONE  
3  NO ACTION  NONE  
4  NO ACTION  NONE  


In [19]:
cursor.execute('SELECT name FROM sqlite_master where type="table"')
print(cursor.fetchall())

[('patients',), ('careplans',), ('conditions',), ('observations',), ('patients_',), ('observations_',), ('careplans_code',), ('careplans_reasoncode',), ('conditions_',), ('fact_table',)]


## Analysis

In [20]:
print(pd.read_sql_query("SELECT reasoncode FROM careplans_reasoncode WHERE reasondescription='Diabetes'", connection))

   REASONCODE
0    44054006


In [21]:
print(pd.read_sql_query('''SELECT fact_table.patient_ID
                        FROM fact_table
                        WHERE careplans_reasoncode=
                        (SELECT reasoncode FROM careplans_reasoncode WHERE reasondescription='Diabetes' OR reasondescription='Prediabetes')
                        ''', connection))

                               PATIENT_ID
0    dea160e9-e636-5716-bb5a-260a9a338fea
1    44428a2f-14c3-8077-f48b-bae301cc7734
2    0ec97729-9fe5-f6be-6a05-2a3d30f705a1
3    14e0d9b0-b5fb-6f3f-7f2f-3170f42e6dc9
4    7071fb3c-05a8-260f-31fe-d2691b9b0d47
..                                    ...
468  7805af3c-2d62-ec81-ae34-db3b7c7a2b8c
469  16e3dc05-c239-4e63-ea9f-3df0427dcdd9
470  24cf50a2-1e8f-4936-a025-5024bb2fd7fe
471  d98c4299-a3f0-337b-c143-7866dbca6309
472  b1f4c342-6e1a-d0ea-f402-9395e395e158

[473 rows x 1 columns]


## Aufräumen & Logs speichern

In [22]:
connection.close()
#logger.logTimings()
#logger.writeToFile("../artefacts-for-release/analysis-log.json")